# PM Type Analysis and Excel Report Generation

## Import Libraries and Load Configuration

In [1]:
import pandas as pd
import openpyxl
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo
import re
import logging
import yaml
import os

# Load configuration
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


## Define Helper Functions

In [2]:
def get_pm_type(row):
    # Get values from the row
    type_value = row['Type']
    actual_pertain_value = row['Actual Pertain']
    
    # Case 1: If both Type and Actual Pertain are empty (NaN)
    if pd.isna(type_value) and pd.isna(actual_pertain_value):
        return 'Others'
    
    # Look up the PM type from TYPE_MAP, default to 'Others' if not found
    type_pm = config['TYPE_MAP'].get(type_value, 'Others')
    actual_pertain_pm = config['TYPE_MAP'].get(actual_pertain_value, 'Others')
    
    # Priority order for determining PM type:
    
    # Priority 1: If Actual Pertain maps to something specific (not Base-Build or Others)
    if actual_pertain_pm not in ['Base-Build', 'Others']:
        return actual_pertain_pm
        
    # Priority 2: If Type maps to something specific (not Base-Build or Others)
    elif type_pm not in ['Base-Build', 'Others']:
        return type_pm
        
    # Priority 3: If Type is Base-Build
    elif type_pm == 'Base-Build':
        return type_pm
        
    # Priority 4: Default case - return Others
    else:
        return 'Others'

def map_category(category):
    if pd.isna(category) or category == 0 or category == '':
        return 'Others'
    elif category.startswith('Others'):
        return 'Others'
    elif category in config['CORRECT_CATEGORY_ORDER']:
        return category
    else:
        return 'Others'

def auto_adjust_column_width(worksheet):
    for column in worksheet.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2
        worksheet.column_dimensions[column_letter].width = adjusted_width

def format_as_table(worksheet, data_range):
    valid_name = re.sub(r'[^\w]', '_', worksheet.title)
    table_name = f"Table_{valid_name}"
    table = Table(displayName=table_name, ref=data_range)
    style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                           showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    table.tableStyleInfo = style
    worksheet.add_table(table)

def add_total_row(worksheet):
    last_row = worksheet.max_row
    last_col = worksheet.max_column
    worksheet.cell(row=last_row + 1, column=1, value='Total').font = Font(bold=True)
    for col in range(2, last_col + 1):
        col_letter = get_column_letter(col)
        total_formula = f'=SUM({col_letter}2:{col_letter}{last_row})'
        cell = worksheet.cell(row=last_row + 1, column=col, value=total_formula)
        cell.font = Font(bold=True)
        cell.number_format = '_($* #,##0.00_);_($* (#,##0.00);_($* "-"??_);_(@_)'

def calculate_tsmc_po_total(updated_po_data, pm_type):
    return updated_po_data[updated_po_data['Main Page'] == pm_type]['Total Contract $'].sum()

## Data Loading and Processing

In [3]:
def load_and_process_data():
    # Load pc_overview data
    pc_overview = pd.read_excel(config['EXISTING_FILE_PATH'], sheet_name='pc_overview')
    
    # Process data
    pc_overview = pc_overview[pc_overview['TSMC 新工'] == 'Base-Build']
    pc_overview['PM Type'] = pc_overview.apply(get_pm_type, axis=1)
    pc_overview['Mapped_Category'] = pc_overview['Category'].apply(map_category)
    pc_overview = pc_overview[pc_overview['Amount'].notna() & (pc_overview['Amount'] != 0)]
    
    # Define the columns in the desired order
    columns_order = [
        'TSMC 新工',
        'PM Type',
        'Mapped_Category',
        'Scope',
        'Project Number',
        'PO #',
        'Amount',
        'Project Name',
        'PO Description',
        'Vendor/Subcontractor',
        'Accumulated AP (Paid)',
        'AP %',
        'Category',
        'Main/CO/DCR',
        'Actual Pertain',
        'Type',
        'Type2'
    ]
    
    # Reorder the columns
    pc_overview = pc_overview[columns_order]
    
    # Load updated PO data (New Filtered CO Settlement)
    updated_po_data = pd.read_excel(config['UPDATED_PO_DATA_PATH'], sheet_name="Updated PO Data")
    
    return pc_overview, updated_po_data


## Generate Reports


In [4]:
def create_pc_overview_sheet(writer, pc_overview, sheet_name='pc_overview AP'):
    pc_overview.to_excel(writer, sheet_name=sheet_name, index=False)
    worksheet = writer.sheets[sheet_name]
    
    # Set custom column widths
    custom_widths = {
        'TSMC 新工': 20,
        'PM Type': 20,
        'Mapped_Category': 20,
        'Scope': 20,
        'Project Number': 20,
        'PO #': 20,
        'Amount': 20,
        'Project Name': 55,
        'PO Description': 90
    }
    
    for col_name, width in custom_widths.items():
        if col_name in pc_overview.columns:
            col_letter = get_column_letter(pc_overview.columns.get_loc(col_name) + 1)
            worksheet.column_dimensions[col_letter].width = width
    
    # Auto-adjust other column widths
    for idx, column in enumerate(worksheet.columns):
        if worksheet.column_dimensions[get_column_letter(idx + 1)].width is None:
            max_length = 0
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            worksheet.column_dimensions[get_column_letter(idx + 1)].width = adjusted_width
    
    format_as_table(worksheet, f"A1:{get_column_letter(worksheet.max_column)}{worksheet.max_row}")

def get_ar_column_letters(ar_sheet):
    contract_col = main_page_col = None
    for idx, cell in enumerate(ar_sheet[1], 1):
        if cell.value == 'Total Contract $':
            contract_col = get_column_letter(idx)
        elif cell.value == 'Main Page':
            main_page_col = get_column_letter(idx)
    return contract_col, main_page_col

def create_combined_sheet(writer, pc_overview, updated_po_data):
    combined_sheet = writer.book.create_sheet(title="Combined PM Types")
    headers = ["Category", "Amount", "TSMC PO Total $"]
    row_offset = 1

    # Get dynamic column letters
    ar_sheet = writer.sheets['pc_overview AR']
    contract_col, main_page_col = get_ar_column_letters(ar_sheet)

    # Get the column letters for PM Type, Mapped_Category, and Amount
    pm_type_col = get_column_letter(pc_overview.columns.get_loc('PM Type') + 1)
    mapped_category_col = get_column_letter(pc_overview.columns.get_loc('Mapped_Category') + 1)
    amount_col = get_column_letter(pc_overview.columns.get_loc('Amount') + 1)

    for pm_type in pc_overview['PM Type'].unique():
        pm_data = pc_overview[pc_overview['PM Type'] == pm_type].copy()
        
        combined_sheet.cell(row=row_offset, column=1, value=pm_type).font = Font(bold=True)
        
        for col, header in enumerate(headers, start=1):
            combined_sheet.cell(row=row_offset + 1, column=col, value=header).font = Font(bold=True)
        
        for idx, category in enumerate(config['CORRECT_CATEGORY_ORDER']):
            excel_row = row_offset + idx + 2
            combined_sheet.cell(row=excel_row, column=1, value=category)
            
            # Use SUMIFS formula with dynamic column letters
            amount_formula = f"=SUMIFS('pc_overview AP'!${amount_col}:${amount_col}, 'pc_overview AP'!${pm_type_col}:${pm_type_col}, \"{pm_type}\", 'pc_overview AP'!${mapped_category_col}:${mapped_category_col}, \"{category}\")"
            combined_sheet.cell(row=excel_row, column=2, value=amount_formula)
        
        total_row = row_offset + len(config['CORRECT_CATEGORY_ORDER']) + 2
        combined_sheet.cell(row=total_row, column=1, value="Total").font = Font(bold=True)
        
        # Amount total
        amount_total_formula = f'=SUM(B{row_offset + 2}:B{total_row - 1})'
        combined_sheet.cell(row=total_row, column=2, value=amount_total_formula).font = Font(bold=True)
        
        # TSMC PO Total $ with dynamic column letters
        tsmc_po_formula = f"=SUMIFS('pc_overview AR'!${contract_col}:${contract_col}, 'pc_overview AR'!${main_page_col}:${main_page_col}, \"{pm_type}\")"
        combined_sheet.cell(row=total_row, column=3, value=tsmc_po_formula).font = Font(bold=True)
        
        for col in range(2, len(headers) + 1):
            for row in range(row_offset + 2, total_row + 1):
                combined_sheet.cell(row=row, column=col).number_format = '_($* #,##0.00_);_($* (#,##0.00);_($* "-"??_);_(@_)'
        
        table_range = f'A{row_offset + 1}:{get_column_letter(len(headers))}{total_row}'
        table_name = f'Table_{re.sub(r"[^a-zA-Z0-9]", "_", pm_type)}_{row_offset}'
        table = Table(displayName=table_name, ref=table_range)
        style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                             showLastColumn=False, showRowStripes=True, showColumnStripes=False)
        table.tableStyleInfo = style
        combined_sheet.add_table(table)
        
        row_offset = total_row + 3

    auto_adjust_column_width(combined_sheet)
    combined_sheet.column_dimensions['B'].width = 20
    combined_sheet.column_dimensions['C'].width = 20
    
def create_detailed_combined_sheet(writer, pc_overview, updated_po_data):
    detailed_sheet = writer.book.create_sheet(title="Detailed Combined PM Types")
    headers = ["Category", "Scope", "Amount", "TSMC PO Total $"]
    row_offset = 1

    # Get dynamic column letters from AR sheet for the SUMIFS formula
    ar_sheet = writer.sheets['pc_overview AR']
    contract_col, main_page_col = get_ar_column_letters(ar_sheet)

    # Get the column letters for PM Type, Mapped_Category, Scope, and Amount
    pm_type_col = get_column_letter(pc_overview.columns.get_loc('PM Type') + 1)
    mapped_category_col = get_column_letter(pc_overview.columns.get_loc('Mapped_Category') + 1)
    scope_col = get_column_letter(pc_overview.columns.get_loc('Scope') + 1)
    amount_col = get_column_letter(pc_overview.columns.get_loc('Amount') + 1)

    for pm_type in pc_overview['PM Type'].unique():
        pm_data = pc_overview[pc_overview['PM Type'] == pm_type].copy()
        
        detailed_sheet.cell(row=row_offset, column=1, value=pm_type).font = Font(bold=True)
        row_offset += 1
        
        for col, header in enumerate(headers, start=1):
            detailed_sheet.cell(row=row_offset, column=col, value=header).font = Font(bold=True)
        row_offset += 1
        
        start_row = row_offset
        
        for category in config['CORRECT_CATEGORY_ORDER']:
            category_data = pm_data[pm_data['Mapped_Category'] == category]
            
            if not category_data.empty:
                scope_sums = category_data.groupby('Scope')['Amount'].sum().reset_index()
                
                for _, row in scope_sums.iterrows():
                    detailed_sheet.cell(row=row_offset, column=1, value=category)
                    detailed_sheet.cell(row=row_offset, column=2, value=row['Scope'])
                    
                    # Create the SUMIFS formula
                    amount_formula = f"=SUMIFS('pc_overview AP'!${amount_col}:${amount_col}, 'pc_overview AP'!${pm_type_col}:${pm_type_col}, \"{pm_type}\", 'pc_overview AP'!${mapped_category_col}:${mapped_category_col}, \"{category}\", 'pc_overview AP'!${scope_col}:${scope_col}, \"{row['Scope']}\")"
                    detailed_sheet.cell(row=row_offset, column=3, value=amount_formula)
                    
                    row_offset += 1
            else:
                detailed_sheet.cell(row=row_offset, column=1, value=category)
                detailed_sheet.cell(row=row_offset, column=2, value="N/A")
                detailed_sheet.cell(row=row_offset, column=3, value=0)
                row_offset += 1
        
        # Add Total row
        total_row = row_offset
        detailed_sheet.cell(row=total_row, column=1, value="Total").font = Font(bold=True)
        for col in range(2, len(headers) + 1):
            cell = detailed_sheet.cell(row=total_row, column=col)
            cell.font = Font(bold=True)
            if col == 3:
                col_letter = get_column_letter(col)
                formula = f'=SUM({col_letter}{start_row}:{col_letter}{total_row-1})'
                cell.value = formula
            elif col == 4:
                # Replace static value with SUMIFS formula
                tsmc_po_formula = f"=SUMIFS('pc_overview AR'!${contract_col}:${contract_col}, 'pc_overview AR'!${main_page_col}:${main_page_col}, \"{pm_type}\")"
                cell.value = tsmc_po_formula
        
        # Apply table style
        table_range = f'A{start_row-1}:{get_column_letter(len(headers))}{total_row}'
        table_name = f'Table_{re.sub(r"[^a-zA-Z0-9]", "_", pm_type)}_{start_row}'
        table = Table(displayName=table_name, ref=table_range)
        style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                               showLastColumn=False, showRowStripes=True, showColumnStripes=False)
        table.tableStyleInfo = style
        detailed_sheet.add_table(table)
        
        row_offset = total_row + 2

    # Add the From APN table
    row_offset += 1  # Add one row of spacing

    # Read the From APN data from the summary_tables_merged.xlsx file
    file_path = os.path.join('APN_Files', 'summary_tables_merged.xlsx')
    from_apn_df = pd.read_excel(file_path, sheet_name="From APN", header=1)
    from_apn_df = from_apn_df.iloc[:, :2]  # Keep only the first two columns

    # Write the From APN table
    detailed_sheet.cell(row=row_offset, column=1, value="From APN").font = Font(bold=True)
    row_offset += 1

    start_row = row_offset
    for col, header in enumerate(['Scope', 'Amount'], start=1):
        detailed_sheet.cell(row=row_offset, column=col, value=header).font = Font(bold=True)
    row_offset += 1

    for _, row in from_apn_df.iterrows():
        for col, value in enumerate(row, start=1):
            cell = detailed_sheet.cell(row=row_offset, column=col, value=value)
            if isinstance(value, (int, float)):
                cell.number_format = '_($* #,##0.00_);_($* (#,##0.00);_($* "-"??_);_(@_)'
        row_offset += 1

    # Add Total row
    detailed_sheet.cell(row=row_offset, column=1, value="Total").font = Font(bold=True)
    total_formula = f'=SUM(B{start_row + 1}:B{row_offset - 1})'
    total_cell = detailed_sheet.cell(row=row_offset, column=2, value=total_formula)
    total_cell.font = Font(bold=True)
    total_cell.number_format = '_($* #,##0.00_);_($* (#,##0.00);_($* "-"??_);_(@_)'
    row_offset += 1

    # Apply table style for From APN table
    table_range = f'A{start_row}:{get_column_letter(len(from_apn_df.columns))}{row_offset-1}'
    table_name = f'Table_From_APN'
    table = Table(displayName=table_name, ref=table_range)
    style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                           showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    table.tableStyleInfo = style
    detailed_sheet.add_table(table)

    # Remove the original From APN sheet
    if "From APN" in writer.book.sheetnames:
        del writer.book["From APN"]

    for col in range(3, len(headers) + 1):
        for row in range(1, row_offset):
            detailed_sheet.cell(row=row, column=col).number_format = '_($* #,##0.00_);_($* (#,##0.00);_($* "-"??_);_(@_)'

    # Adjust column widths at the end
    auto_adjust_column_width(detailed_sheet)
    
    # Set specific widths for columns C and D
    detailed_sheet.column_dimensions['C'].width = 20
    detailed_sheet.column_dimensions['D'].width = 20

def create_base_build_breakdown(writer, pc_overview):
    base_build_data = pc_overview[pc_overview['PM Type'] == 'Base-Build'].copy()
    base_build_grouped = base_build_data.groupby(['Scope', 'Type2'])['Amount'].sum().reset_index()
    base_build_grouped = base_build_grouped.sort_values(['Scope', 'Amount'], ascending=[True, False])
    base_build_grouped.to_excel(writer, sheet_name='Base-Build_breakdown', index=False)
    worksheet = writer.sheets['Base-Build_breakdown']
    auto_adjust_column_width(worksheet)
    format_as_table(worksheet, f"A1:{get_column_letter(worksheet.max_column)}{worksheet.max_row}")
    add_total_row(worksheet)

def generate_reports(pc_overview, updated_po_data):
    with pd.ExcelWriter(config['NEW_FILE_PATH'], engine='openpyxl') as writer:
        create_pc_overview_sheet(writer, pc_overview, sheet_name='pc_overview AP')
        create_pc_overview_sheet(writer, updated_po_data, sheet_name='pc_overview AR')
        create_combined_sheet(writer, pc_overview, updated_po_data)
        create_detailed_combined_sheet(writer, pc_overview, updated_po_data)
        create_base_build_breakdown(writer, pc_overview)


## Cleanup Workbook

In [5]:
def cleanup_workbook():
    wb = openpyxl.load_workbook(config['NEW_FILE_PATH'])
    sheets_to_keep = ['pc_overview AP', 'pc_overview AR', 'Combined PM Types', 'Detailed Combined PM Types', 'Base-Build_breakdown']
    for sheet_name in wb.sheetnames:
        if sheet_name not in sheets_to_keep:
            del wb[sheet_name]
    wb.save(config['NEW_FILE_PATH'])

# Main execution
if __name__ == "__main__":
    # Load and process data
    pc_overview, updated_po_data = load_and_process_data()

    # Generate reports
    generate_reports(pc_overview, updated_po_data)

    # Cleanup workbook
    cleanup_workbook()

    print(f"Workbook saved with only required sheets.")


Workbook saved with only required sheets.
